In [1]:
import subprocess
import os
from datetime import datetime, timezone, timedelta

In [2]:
def Get_ADB_Serial():
    result = subprocess.run(
        ['adb', 'get-serialno'],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    serial = result.stdout.strip()
    if serial and serial != 'unknown':
        return serial
    return None

In [3]:
def List_Filepaths(extensions = ['.jpg', '.jpeg', '.png', '.mp4'], source = '/sdcard/' ):
    all_files = []
    for ext in extensions:
        cmd = ['adb', 'shell', 'find', f'{source} -iname "*{ext}"']
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            found_files = result.stdout.strip().split('\n')
            all_files.extend([f.strip() for f in found_files if f.strip()])
        else:
            print(f"Error finding *{ext} files:", result.stderr)
    return all_files

In [4]:
def Get_File_Modtime(file_path):
    cmd = ['adb', 'shell', f'stat -c %Y "{file_path}"']
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0 or not result.stdout.strip().isdigit():
        return None
    return int(result.stdout.strip())  

In [5]:
def Pull_Files(filepaths, day : int, month : int, serial : str):
    os.makedirs(f'Dump/{serial}', exist_ok=True)
    
    for path in filepaths:
        filename = f'{month}_{day}_{serial}_{os.path.basename(path)}'
        local_path = os.path.join(f'Dump/{serial}', filename)
        print(filename)
        subprocess.run(['adb', 'pull', path, local_path])

In [6]:
def Nuke():
    cmd = ['adb', 'shell', 'rm', '-rf', '/sdcard/']
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print(result.stdout)

## Const Search Time Range

In [7]:
PHT = timezone(timedelta(hours=8))

start_date_pht = datetime(2025, 7, 1, 0, 0, 0, tzinfo=PHT)
end_date_pht   = datetime(2025, 7, 30, 23, 59, 59, tzinfo=PHT)

start_date_utc = start_date_pht.astimezone(timezone.utc)
end_date_utc = end_date_pht.astimezone(timezone.utc)

## Serial Number

In [8]:
serial = Get_ADB_Serial()
assert serial
print("🔢 ADB Serial Number:", serial)

🔢 ADB Serial Number: R5CN71E9HYH


## Raw Filepaths

In [9]:
print(f"Scanning phone for files...")
# found_filepaths = List_Filepaths(extensions=['.jpg', 'jpeg'], source='/sdcard/Download/')

found_filepaths = List_Filepaths(source='/sdcard/DCIM/')
print(f"Found {len(found_filepaths)} files.")

Scanning phone for files...
Found 949 files.


## Filtered Filepaths

In [10]:
print(f"Filtering by date range (PHT): {start_date_pht} → {end_date_pht}")
filtered_by_date = dict()

# FILTER FILTEPATHS BY DATE
for path in found_filepaths:
    modtime = Get_File_Modtime(path)
    assert modtime
    
    mod_date = datetime.fromtimestamp(modtime, timezone.utc)

    if start_date_utc <= mod_date <= end_date_utc:
    # DEVICES DIDN'T HAVE DATES CONFIGURED. EXTRACT EVERYDAY INSTEAD 
    # if True:
        # ADD BY DATE
        if mod_date.date() not in filtered_by_date:
            filtered_by_date[mod_date.date()] = []
        filtered_by_date[mod_date.date()].append(path)


Filtering by date range (PHT): 2025-07-01 00:00:00+08:00 → 2025-07-30 23:59:59+08:00


# Sorted Extract

In [11]:
for date in filtered_by_date.keys():
    print(f'Pulling {len(filtered_by_date[date])} files from date {date}')
    Pull_Files(filepaths = filtered_by_date[date], serial = serial, day = date.day, month = date.month)
print("✅ Done.")

Pulling 14 files from date 2025-07-15
7_15_R5CN71E9HYH_20250715_095901.jpg
7_15_R5CN71E9HYH_20250715_095858.jpg
7_15_R5CN71E9HYH_20250715_100510.jpg
7_15_R5CN71E9HYH_20250715_131320.jpg
7_15_R5CN71E9HYH_20250715_163741.jpg
7_15_R5CN71E9HYH_20250715_095707.jpg
7_15_R5CN71E9HYH_20250715_095903.jpg
7_15_R5CN71E9HYH_20250715_131330.jpg
7_15_R5CN71E9HYH_20250715_095703.jpg
7_15_R5CN71E9HYH_20250715_095712.jpg
7_15_R5CN71E9HYH_20250715_100630.jpg
7_15_R5CN71E9HYH_Screenshot_20250715_091202_Gmail.jpg
7_15_R5CN71E9HYH_Screenshot_20250715_142727_Gmail.jpg
7_15_R5CN71E9HYH_Screenshot_20250715_142752_Gmail.jpg
Pulling 3 files from date 2025-07-03
7_3_R5CN71E9HYH_20250703_215441.jpg
7_3_R5CN71E9HYH_20250703_215415.jpg
7_3_R5CN71E9HYH_20250703_215411.jpg
Pulling 8 files from date 2025-07-01
7_1_R5CN71E9HYH_20250701_121722.jpg
7_1_R5CN71E9HYH_20250701_215105.jpg
7_1_R5CN71E9HYH_20250701_121832.jpg
7_1_R5CN71E9HYH_20250701_121711.jpg
7_1_R5CN71E9HYH_20250701_215116.jpg
7_1_R5CN71E9HYH_20250701_215027